# 개요

* 안드로이드 앱은 핸드폰의 제한된 공간에서 작동 되는데 앱 외부에 있는 리소스(카메라, SMS, 저장공간 등)에 접속하기 위해서 핸드폰 소유자의 접근 허가 권한을 받아야 이용이 가능하다. Permission이란 안드로이드 애플리케이션이 특정 기능을 수행할 수 있는 권한(rights)을 요구하는 것임
* 안드로이드 마쉬멜로우 이전 버전에서는 코드상에 권한허가를 넣어주면 권한 획득이 가능했는데 마쉬멜로우 버전 이후부터는 사용자가 직접 권한을 허락해야 사용이 가능하도록 바뀜
* 안드로이드 6.0 버전부터 사용자 기밀 데이터에 접근할 수 있는 일부 Dangerous permission에 대해서 코드 상에 관련 내용을 서술하고 동적으로 권한을 요청하도록 명시하고 있으나 반드시 AndroidManifest.xml에도 작성해야 함.
* 반드시 AndroidManifest.xml에 선언되므로 정적 분석으로도 충분히 추출 가능

# 추출방법

## aapt 사용

In [11]:
import os, subprocess
terminal = 'aapt d permissions /mnt/c/Users/junve/Desktop/apk/ap.apk > ./result.txt'

# 작성일(23.01.12) 기준 os 라이브러리 방식은 보안상의 단점과, BOF문제가 존재하므로 subprocess 추천
# os 라이브러리
os.system(terminal)

# subprocess 라이브러리
result = subprocess.run(terminal, shell = True)

## androguard 사용

In [15]:
from androguard.misc import AnalyzeAPK
a, *_ = AnalyzeAPK('/home/ghwns82/ap.apk')
a.permissions

['android.permission.ACCESS_NETWORK_STATE',
 'android.permission.WRITE_EXTERNAL_STORAGE',
 'android.permission.INTERNET']

In [17]:
a.get_permissions()

['android.permission.ACCESS_NETWORK_STATE',
 'android.permission.WRITE_EXTERNAL_STORAGE',
 'android.permission.INTERNET']

# Dangerous permission 선정방법

## Deverloper perm-info 웹스크래핑

## Dangerous lv 선별